In [ ]:
import h5py
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sn
import time
import sys
######################################################################################
######################################################################################
################################    Question 1     #####################################
######################################################################################
######################################################################################

class Autoencoder_NN(object):
    """
    Autoencoder class for Question 1
    class for creating an autoencoder neural network, which is a type of neural network that learns to reconstruct 
    it inputs by learning a compressed representation of the data through the encoding part of the network and 
    then learning to reconstruct the original data from this compressed representation through the decoding part 
    of the network.
    """


    def init_coeff(self, Lin, Lhid):
        """
        The method takes in two arguments: Lin which is the size of the input layer and Lhid which is the size of the hidden layer.
        The method initializes the weights using a uniform distribution with a range 
        that is calculated using the formula sqrt(6 / (Lin + Lhid)). The method also initializes the biases for the hidden and output 
        layers using the same formula.

        """

        L = Lin

        r = np.sqrt(6 / (Lin + Lhid))
        W1 = np.random.uniform(-r, r, size=(Lin, Lhid))
        b1 = np.random.uniform(-r, r, size=(1, Lhid))

        r = np.sqrt(6 / (Lhid + L))
        W2 = W1.T
        b2 = np.random.uniform(-r, r, size=(1, L))

        Mul = (W1, W2, b1, b2)
        tMul = (0, 0, 0, 0)

        return Mul, tMul


    def train(self, data, coeff, eta=0.1, alpha=0.9, epoch=10, batch_size=None):
        """
       data: the training data
       
       coeff: a dictionary containing the required parameters for the autoencoder
       
       eta: the learning rate, which determines how fast the model updates its weights
       
       alpha: the momentum multiplier, which determines how much the model should consider 
       its past weight updates when making new updates
       
       epoch: the number of times the model should go through the training data
       
       batch_size: the size of the batches to use when training the model with stochastic gradient descent (SGD). 
       If batch_size is None, then the entire training data will be used in each iteration.

        """

        J_array = []
        if batch_size is None:
            batch_size = data.shape[0]

        Lin = coeff["Lin"]
        Lhid = coeff["Lhid"]
        Mul, tMul = self.init_coeff(Lin,  Lhid)

        iter_per_epoch = int(data.shape[0] / batch_size)
        """
        function first initializes the weights and momentum term for the weights, 
        then runs through a loop for the specified number of epochs. In each epoch, 
        the training data is shuffled and divided into batches. 

        """
        for i in range(epoch):

            time_start = time.time()

            J_total = 0

            start = 0
            end = batch_size

            p = np.random.permutation(data.shape[0])
            data = data[p]

            tMul = (0, 0, 0, 0)

            for j in range(iter_per_epoch):

                batchData = data[start:end]

                J, Jgrad, cache = self.aeCost(Mul, batchData, coeff)
                Mul, tMul = self.solver(Jgrad, cache, Mul, tMul, eta, alpha)

                J_total += J
                start = end
                end += batch_size

      

            J_total = J_total/iter_per_epoch

            print("Amount_loss: {:.2f} [Epoch {} of {}]".format(J_total, i+1, epoch ))
            J_array.append(J_total)

        print("\n")

        return Mul, J_array


    def aeCost(self, Mul, data, coeff):
        """
        function that calculates the cost and the gradients of the weights for an autoencoder neural network.
        In this function, the reconstruction error is calculated as the mean squared error between the input 
        data and the output of the autoencoder, which is obtained by passing the input data through the encoder
        and the decoder.
        """

        N = data.shape[0]

        W1, W2, b1, b2 = Mul

        rho = coeff["rho"]
        beta = coeff["beta"]
        lmb = coeff["lmb"]
        Lin = coeff["Lin"]
        Lhid = coeff["Lhid"]

        u = data @ W1 + b1
        h, h_drv = self.sigmoid(u)  # N x Lhid
        # h, h_drv = self.tanh(u)  # N x Lhid
        v = h @ W2 + b2
        o, o_drv = self.sigmoid(v)  # N x Lin
        # o, o_drv = self.tanh(v)  # N x Lin

        rho_b = h.mean(axis=0, keepdims=True)  # 1 x Lhid

        """
        The function also includes a regularization term, called Tikhonov regularization, 
        which is added to the cost to prevent overfitting. This term is calculated as the 
        sum of the squares of the weights, multiplied by a regularization parameter lmb.
        Additionally, the function includes a term called the Kullback-Leibler divergence, 
        which measures the difference between the distribution of the activations of the 
        hidden layer and a target distribution with mean rho. This term is multiplied by 
        a parameter beta to control its importance in the overall cost.
        """



        Amount_loss = 0.5/N * (np.linalg.norm(data - o, axis=1) ** 2).sum()
        tykhonov = 0.5 * lmb * (np.sum(W1 ** 2) + np.sum(W2 ** 2))
        kullback = rho * np.log(rho/rho_b) + (1 - rho) * np.log((1 - rho)/(1 - rho_b))
        kullback = beta * kullback.sum()

        J = Amount_loss + tykhonov + kullback

        Loss = -(data - o)/N
        tyk2 = lmb * W2
        tyk1 = lmb * W1
        kullback = beta * (- rho/rho_b + (1-rho)/(1 - rho_b))/N

        cache = (data, h, h_drv, o_drv)
        Jgrad = (Loss, tyk2, tyk1, kullback)

        """
        The function returns the cost J, the gradients of the weights Jgrad, and some
        intermediate variables stored in a cache. The gradients of the weights are 
        used to update the weights in the optimization process.
        """
        return J, Jgrad, cache


    def solver(self, Jgrad, cache, Mul, tMul, eta, alpha):
        """
       function that performs gradient descent on a neural network with two layers (one hidden and one output). It takes as input:

       Jgrad: a tuple containing the error gradients for the loss, tyk2, tyk1, and kullback.
       cache: a tuple containing the data, h, h_drv, and o_drv values that are needed to compute the updates.
       Mul: a tuple containing the weights and biases for the two layers (W1, W2, b1, b2).
       tMul: a tuple containing the corresponding momentum terms for the weights and biases (mW1, mW2, mb1, mb2).
       eta: the learning rate.
       alpha: the momentum multiplier.
        """

        W1, W2, b1, b2 = Mul

        dW1 = 0
        dW2 = 0
        db1 = 0
        db2 = 0

        data, h, h_drv, o_drv = cache
        Loss, tyk2, tyk1,  kullback = Jgrad

        delta = Loss * o_drv


        dW2 = h.T @ delta + tyk2
        db2 = delta.sum(axis=0, keepdims=True)

        delta = h_drv * (delta @ W2.T + kullback)

        dW1 = data.T @ delta + tyk1
        db1 = delta.sum(axis=0, keepdims=True)

      
        dW2 = (dW1.T + dW2)/2
        dW1 = dW2.T

        dMul = (dW1, dW2, db1, db2)

        """
        The function then computes the updates for the weights and biases using 
        the error gradients and the values in the cache tuple. It then calls the
        update function to update the weights and biases and returns the 
        updated values for Mul and tMul.

        """


        Mul, tMul = self.update(Mul, tMul, dMul, eta, alpha)

        return Mul, tMul


    def update(self, Mul, tMul, dMul, eta, alpha):
        """
       The update function takes as input:

       Mul: a tuple containing the weights and biases for the two layers (W1, W2, b1, b2).

       tMul: a tuple containing the corresponding momentum terms for the weights 
       and biases (mW1, mW2, mb1, mb2).
       
       dMul: a tuple containing the updates for the weights and biases (dW1, dW2, db1, db2).
       
       eta: the learning rate.
       
       alpha: the momentum multiplier.

        """

        W1, W2, b1, b2 = Mul
        dW1, dW2, db1, db2 = dMul
        mW1, mW2, mb1, mb2 = tMul

        mW1 = eta * dW1 + alpha * mW1
        mW2 = eta * dW2 + alpha * mW2
        mb1 = eta * db1 + alpha * mb1
        mb2 = eta * db2 + alpha * mb2

        W1 -= mW1
        W2 -= mW2
        b1 -= mb1
        b2 -= mb2
        """
        The function first computes the updated momentum terms for each weight 
        and bias using the formula mW = eta * dW + alpha * mW, where mW is the 
        momentum term, dW is the update for the weight, eta is the learning rate, 
        and alpha is the momentum multiplier. It then updates the weights and 
        biases using the formula W -= mW, where W is the weight and mW is the 
        corresponding momentum term. Finally, it returns the updated values 
        for Mul and tMul.

        """

        assert (W1 == W2.T).all()
        Mul = (W1, W2, b1, b2)
        tMul = (mW1, mW2, mb1, mb2)

        return Mul, tMul


    def predict(self, data, We):
        """
       function that performs a forward pass through a neural network with two layers (one hidden and one output). It takes as input:

       data: the input data for the neural network.
       Mul: a tuple containing the weights and biases for the two layers (W1, W2, b1, b2).
        """

        W1, W2, b1, b2 = Mul

        u = data @ W1 + b1
        h = self.sigmoid(u)[0]
        v = h @ W2 + b2
        o = self.sigmoid(v)[0]
        return o


    def sigmoid(self, X):
        """
        The function first computes the sigmoid activation for each element in X 
        using the formula a = 1 / (1 + np.exp(-X)), where np.exp is the exponentiation 
        function from the NumPy library. It then computes the derivative of the sigmoid 
        function for each element in a using the formula d = a * (1 - a). 
        Finally, it returns both a and d as a tuple.
        """
        a = 1 / (1 + np.exp(-X))
        d = a * (1 - a)
        return a, d



def normalize(X):


    return (X - X.min())/(X.max() - X.min())


def plot(W, name, dim1, dim2):
    """
    W: a matrix of weights to be plotted.
    name: a string containing the filename for the plot.
    dim1: the width of the grid of images.
    dim2: the height of the grid of images.
    """

    fig, ax = plt.subplots(dim2, dim1, figsize=(dim1, dim2), dpi=320, facecolor='w', edgecolor='k')
    k = 0
    for i in range(dim2):
        for j in range(dim1):
            ax[i, j].imshow(W[k], cmap='gray')
            ax[i, j].axis("off")
            k += 1

    fig.subplots_adjust(wspace=0.1, hspace=0.1, left=0, right=1, bottom=0, top=1)
    fig.savefig(name + ".png")
    plt.close(fig)


def q1():
    """
      To perform image processing on a dataset and then train an autoencoder 
    neural network on the processed data. It does the following:

      Opens a file called data1.h5 using the h5py library, reads the data stored 
    in the 'data' key, and converts it to a NumPy array of type float64.
      
      Converts the data to grayscale using the luminosity model.
      
      Normalizes the grayscale data by subtracting the mean of each image, 
    clipping the data to 3 standard deviations, and normalizing the data to 
    the range [0, 1].
      
      Maps the data to the range [0.1, 0.9].
      
      Splits the data into a training set and a validation set.
      
      Plots 200 random images from the training set in both RGB and grayscale.
      
      Trains an autoencoder neural network on the training set using the train 
    function from the Autoencoder_NN class, with the specified hyperparameters.
      
      Normalizes the weights of the trained autoencoder and reshapes them into 
    a matrix with the same shape as the original images.
    Calls the plot function to visualize the weights as a grid of images.


    """
    filename = "data1.h5"
    h5 = h5py.File(filename, 'r')
    data = h5['data'][()].astype('float64')
    h5.close()


    # convert to grayscale using the luminosity model
    Luminosity = 0.2126 * data[:, 0] + 0.7152 * data[:, 1] + 0.0722 * data[:, 2]

    # normalize
    assert Luminosity.shape[1] == Luminosity.shape[2]
    dim = Luminosity.shape[1]
    Luminosity = np.reshape(Luminosity, (Luminosity.shape[0], dim ** 2))  # flatten

    Luminosity = Luminosity - Luminosity.mean(axis=1, keepdims=True)  # differentiate per image
    std = np.std(Luminosity)  # find std
    Luminosity = np.clip(Luminosity, - 3 * std, 3 * std)  # clip -+3 std
    Luminosity = normalize(Luminosity)  # normalize to 0 - 1

    Luminosity = 0.1 + Luminosity * 0.8  # map to 0.1 - 0.9
    trainData = Luminosity

    # plot 200 random images
    Luminosity = np.reshape(Luminosity, (Luminosity.shape[0], dim, dim))  # reshape for imshow
    data = data.transpose((0, 2, 3, 1))
    Graph1, Line1 = plt.subplots(10, 20, figsize=(20, 10))
    Graph2, Line2 = plt.subplots(10, 20, figsize=(20, 10), dpi=200, facecolor='w', edgecolor='k')

    for i in range(10):
        for j in range(20):
            k = np.random.randint(0, data.shape[0])

            Line1[i, j].imshow(data[k].astype('float'))
            Line1[i, j].axis("off")

            Line2[i, j].imshow(Luminosity[k], cmap='gray')
            Line2[i, j].axis("off")

    Graph1.subplots_adjust(wspace=0, hspace=0, left=0, right=1, bottom=0, top=1)
    Graph2.subplots_adjust(wspace=0, hspace=0, left=0, right=1, bottom=0, top=1)
    Graph1.savefig("q1a_rgb.png")
    Graph2.savefig("q1a_gray.2.png")
    plt.close("all")

    eta = 0.075
    alpha = 0.85
    epoch = 200
    batch_size = 32
    rho = 0.025
    beta = 2
    lmb = 5e-4
    Lin = trainData.shape[1]
    Lhid = 64

    coeff = {"rho": rho, "beta": beta, "lmb": lmb, "Lin": Lin, "Lhid": Lhid}
    ae = Autoencoder_NN()
    w = ae.train(trainData, coeff, eta, alpha, epoch, batch_size)[0]
    W = normalize(w[0]).T
    W = W.reshape((W.shape[0], dim, dim))
    """
    To demonstrate the ability of an autoencoder to learn useful features from 
    images, which can be visualized by examining the weights of the trained autoencoder.

    """
    name = "rho={:.2f}|beta={:.2f}|eta={:.2f}|alpha={:.2f}|lambda={}|batch={}|Lhid={}".format(rho, beta, eta, alpha, lmb, batch_size, Lhid)
    wdim = int(np.sqrt(W.shape[0]))
    plot(W, name + "weights", wdim, wdim)

